In [1]:
import streamlit as st
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import re
from selenium.webdriver.common.by import By
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
# from openpyxl import Workbook
from bs4 import BeautifulSoup
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
import time
import datetime
import requests
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
from openai import OpenAI

In [10]:
rest_df = pd.read_excel("제주도_restnum.xlsx")
rest_df

,카테고리,지역,업체명,순위,점수(mrr),url,restaurant_num
0,음식점,제주시/함덕,우진해장국,1,0.833,https://m.place.naver.com/restaurant/13418405/...,13418405
1,음식점,제주시/함덕,자매국수,2,0.583,https://m.place.naver.com/restaurant/37060300/...,37060300
2,음식점,제주시/함덕,명진전복,3,0.500,https://m.place.naver.com/restaurant/17032617/...,17032617
3,음식점,제주시/함덕,고집돌우럭 제주공항점,5,0.222,https://m.place.naver.com/restaurant/38265405/...,38265405
4,음식점,제주시/함덕,곰막식당,4,0.277,https://m.place.naver.com/restaurant/31204698/...,31204698
5,음식점,애월/협재,제주광해 애월본점,1,0.666,https://m.place.naver.com/restaurant/137482890...,1374828905
6,음식점,애월/협재,수우동 협재점,2,0.500,https://m.place.naver.com/restaurant/36823910/...,36823910
7,음식점,애월/협재,이춘옥원조고등어쌈밥 제주애월,4,0.250,https://m.place.naver.com/restaurant/19872990/...,19872990
8,음식점,애월/협재,한림칼국수,3,0.416,https://m.place.naver.com/restaurant/34635305/...,34635305
9,음식점,애월/협재,협재칼국수,5,0.222,https://m.place.naver.com/restaurant/37889230/...,37889230


In [16]:
search_url = f"https://m.place.naver.com/restaurant/13418405/review/visitor?entry=ple&reviewSort=recent"

In [18]:
options = Options()
# options.add_argument("--headless")

# BS4 setting for secondary access
session = requests.Session()
headers = {
    "User-Agent": "user value"}

retries = Retry(total=5,
                backoff_factor=0.1,
                status_forcelist=[500, 502, 503, 504])

session.mount('http://', HTTPAdapter(max_retries=retries))

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)
driver.get(search_url)

In [13]:
def get_adr_pn(restaurant_num):
    try:
        search_url = f"https://m.place.naver.com/restaurant/{restaurant_num}"
        options = Options()
        # options.add_argument("--headless")
        
        # BS4 setting for secondary access
        session = requests.Session()
        headers = {
            "User-Agent": "user value"}
        
        retries = Retry(total=5,
                        backoff_factor=0.1,
                        status_forcelist=[500, 502, 503, 504])
        
        session.mount('http://', HTTPAdapter(max_retries=retries))
        
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(search_url)
        driver.implicitly_wait(5)
        html = driver.page_source
        bs = BeautifulSoup(html, 'lxml')
        address = bs.select_one('div.vV_z_>a.PkgBl>span.LDgIH')
        phone_number = bs.select_one('div.vV_z_>span.xlx7Q')
        return address.text,phone_number.text
    except Exception as e:
        print(e)
        return None, None
    
    
        

In [19]:
for idx,row in rest_df.iterrows():
    address,phone_number = get_adr_pn(row['restaurant_num'])
    print(address)
    print(phone_number)
    rest_df.loc[idx,"주소"] = address
    rest_df.loc[idx,"전화번호"] = phone_number

제주 제주시 서사로 11
064-757-3393
제주 제주시 항골남길 46 자매국수
064-746-2222
제주 제주시 구좌읍 해맞이해안로 1282
064-782-9944
제주 제주시 임항로 30
0507-1436-1008
제주 제주시 구좌읍 구좌해안로 64
064-727-5111
제주 제주시 애월읍 애월해안로 867 2층
0507-1312-4789
제주 제주시 한림읍 협재1길 11
064-796-5830
제주 제주시 애월읍 일주서로 7213
0507-1402-9914
제주 제주시 한림읍 한림해안로 141
0507-1442-3371
제주 제주시 한림읍 협재로 3
0507-1335-8107
제주 서귀포시 일주서로 879
0507-1408-1540
제주 서귀포시 색달로 10 연돈
0507-1386-7060
제주 서귀포시 동문동로 2
064-762-8927
제주 서귀포시 예래로 255-18 1층
0507-1321-1741
제주 서귀포시 성산읍 섭지코지로 10
064-783-0987
제주 제주시 동문로6길 4 동문시장 12번 게이트 옆
0507-1414-0750
제주 제주시 조천읍 조함해안로 552-3
064-784-5404
'NoneType' object has no attribute 'text'
None
None
제주 제주시 노형2길 51-3
0507-1311-1653
제주 제주시 연오로 5 1층
010-4925-9377
제주 제주시 애월읍 애월북서길 56 애월당
0507-1492-8504
제주 제주시 한림읍 명월성로 16
0507-1414-2343
제주 제주시 애월읍 일주서로 6707 1층
0507-1346-0559
제주 제주시 한림읍 협재1길 29
0507-1387-0280
제주 서귀포시 안덕면 신화역사로 502 청춘당 꽈배기 서부점
0507-1448-5512
제주 서귀포시 성산읍 고성오조로 47 1층
064-782-3125
제주 서귀포시 보목로64번길 178 1층 보래드 베이커스
0507-1314-1450
제주 서귀포시 성산읍 신양로122번길 17 2F
05

In [22]:
rest_df.to_excel("제주도_restnum_with_adr_pn.xlsx")

In [15]:
rest_df

,카테고리,지역,업체명,순위,점수(mrr),url,restaurant_num,주소,전화번호
0,음식점,제주시/함덕,우진해장국,1,0.833,https://m.place.naver.com/restaurant/13418405/...,13418405,제주 제주시 서사로 11,064-757-3393
1,음식점,제주시/함덕,자매국수,2,0.583,https://m.place.naver.com/restaurant/37060300/...,37060300,제주 제주시 항골남길 46 자매국수,064-746-2222
2,음식점,제주시/함덕,명진전복,3,0.500,https://m.place.naver.com/restaurant/17032617/...,17032617,제주 제주시 구좌읍 해맞이해안로 1282,064-782-9944
3,음식점,제주시/함덕,고집돌우럭 제주공항점,5,0.222,https://m.place.naver.com/restaurant/38265405/...,38265405,제주 제주시 임항로 30,0507-1436-1008
4,음식점,제주시/함덕,곰막식당,4,0.277,https://m.place.naver.com/restaurant/31204698/...,31204698,제주 제주시 구좌읍 구좌해안로 64,064-727-5111
5,음식점,애월/협재,제주광해 애월본점,1,0.666,https://m.place.naver.com/restaurant/137482890...,1374828905,제주 제주시 애월읍 애월해안로 867 2층,0507-1312-4789
6,음식점,애월/협재,수우동 협재점,2,0.500,https://m.place.naver.com/restaurant/36823910/...,36823910,NaN,NaN
7,음식점,애월/협재,이춘옥원조고등어쌈밥 제주애월,4,0.250,https://m.place.naver.com/restaurant/19872990/...,19872990,NaN,NaN
8,음식점,애월/협재,한림칼국수,3,0.416,https://m.place.naver.com/restaurant/34635305/...,34635305,NaN,NaN
9,음식점,애월/협재,협재칼국수,5,0.222,https://m.place.naver.com/restaurant/37889230/...,37889230,NaN,NaN


In [27]:

def get_review(restaurant_num):
    try:
        search_url = f"https://m.place.naver.com/restaurant/{restaurant_num}/review/visitor?entry=ple&reviewSort=recent"
        options = Options()
        # options.add_argument("--headless")
    
        # BS4 setting for secondary access
        session = requests.Session()
        headers = {
            "User-Agent": "user value"}
    
        retries = Retry(total=5,
                        backoff_factor=0.1,
                        status_forcelist=[500, 502, 503, 504])
    
        session.mount('http://', HTTPAdapter(max_retries=retries))
    
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=options)
        driver.get(search_url)
    
        driver.implicitly_wait(30)
    
        # Pagedown
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)
    
        try:
            for i in range(20):
                driver.find_element(By.XPATH, '//*[@id="app-root"]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a').click()
                time.sleep(1)
        except Exception as e:
            print('finish')
    
        time.sleep(1)
        html = driver.page_source
        bs = BeautifulSoup(html, 'lxml')
        reviews = bs.select('li.EjjAW')
        row_list = []
        for r in reviews:
            nickname = r.select_one('div.pui__JiVbY3')
            content = r.select_one('div.pui__vn15t2')
            date = r.select_one('div.pui__QKE5Pr>span.pui__gfuUIT>time')
            visit_cnt = r.select('div.pui__QKE5Pr>span.pui__gfuUIT')[1]
    
            # exception handling
            nickname = nickname.text if nickname else ''
            content = content.text if content else ''
            date = date.text if date else ''
            visit_cnt = visit_cnt.text if visit_cnt else ''
    
            time.sleep(0.06)
    
            # print(nickname, '/', content, '/', date)
            row_list.append([nickname, content, date, visit_cnt])
            time.sleep(0.06)
        df = pd.DataFrame(row_list, columns=['nickname', 'content', 'date', 'visit_cnt'])
    
        return df
    except Exception as e:
        print(e)
        return pd.DataFrame(columns=['nickname', 'content', 'date', 'visit_cnt'])

In [28]:
for idx,row in rest_df.iterrows():
    rest_num = row['restaurant_num']
    result_df = get_review(rest_num)
    result_df.to_excel(f"네이버_review_restnum_{rest_num}.xlsx", index=False)

finish
finish
finish
finish
finish
finish
